In [5]:
from typing import List, Tuple
import os
import numpy as np

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset

if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))

torch.manual_seed(4321)
torch.use_deterministic_algorithms(True)

NVIDIA GeForce RTX 3080


## Prepare Dataset

In [3]:
class BCIDataset(Dataset):
    _data_files: List[str]
    _data_loaded: bool = False

    _data: np.ndarray
    _labels: np.ndarray

    def __init__(
        self, data_files: List[str] = ["S4b_train.npz", "X11b_train.npz"]
    ):
        super(BCIDataset, self).__init__()
        for file in data_files:
            if not os.path.exists(file):
                raise FileNotFoundError(f"The data file {file} does not exist.")
        
        self._data_files = data_files

    def __len__(self):
        # lazy loading
        if not self._data_loaded:
            self._load_data()

        return self._labels.shape[0]

    def __getitem__(self, index: int) -> Tuple[torch.Tensor]:
        # lazy loading
        if not self._data_loaded:
            self._load_data()

        return torch.from_numpy(self._data[index]), self._labels[index]

    def _load_data(self):
        data = []
        labels = []

        for file in self._data_files:
            with np.load(file) as f:
                data.append(f["signal"])
                labels.append(f["label"])

        self._data = np.concatenate(data, axis = 0)
        self._labels = np.concatenate(labels, axis = 0)

        self._data = np.expand_dims(self._data, axis=1).swapaxes(-1, -2)
        self._labels -= 1

        mask = np.where(np.isnan(self._data))
        self._data[mask] = np.nanmean(self._data)

        self._data_loaded = True

train_dataset = BCIDataset(data_files = ["S4b_train.npz", "X11b_train.npz"])
test_dataset = BCIDataset(data_files = ["S4b_test.npz", "X11b_test.npz"])

## Build Models

### EEGNet

#### Architecture
![EEGNet](assets/EEGNet.jpg)

In [6]:
class EEGNet(nn.Module):
    first_conv: nn.Sequential
    depthwise_conv: nn.Sequential
    separable_conv: nn.Sequential
    classfier: nn.Sequential

    def __init__(self):
        super(EEGNet, self).__init__()
        self.first_conv = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size = (1, 51), stride = (1, 1), padding = (0, 25), bias = False),
            nn.BatchNorm2d(16, eps = 1e-5, momentum = 0.1, affine = True, track_running_stats = True)
        )
        self.depthwise_conv = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size = (2, 1), stride = (1, 1), groups = 16, bias = False),
            nn.BatchNorm2d(32, eps = 1e-5, momentum = 0.1, affine = True, track_running_stats = True),
            nn.ELU(alpha = 1.0),
            nn.AvgPool2d(kernel_size = (1, 4), stride = (1, 4), padding = 0),
            nn.Dropout(p = 0.25)
        )
        # input: batch, 32, 2, 188
        self.separable_conv = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size = (1, 15), stride = (1, 1), padding = (0, 7), bias = False),
            nn.BatchNorm2d(32, eps = 1e-5, momentum = 0.1, affine = True, track_running_stats = True),
            nn.ELU(alpha = 1.0),
            nn.AvgPool2d(kernel_size = (1, 8), stride = (1, 8), padding = 0),
            nn.Dropout(p = 0.25)
        )
        self.classfier = nn.Sequential(
            nn.Linear(736, 2, bias = True)
        )

    def forward(self, x):
        x = self.first_conv(x)
        x = self.depthwise_conv(x)
        x = self.separable_conv(x)
        return self.classfier(x)

network = EEGNet()
print(network)

EEGNet(
  (first_conv): Sequential(
    (0): Conv2d(1, 16, kernel_size=(1, 51), stride=(1, 1), padding=(0, 25), bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (depthwise_conv): Sequential(
    (0): Conv2d(16, 32, kernel_size=(2, 1), stride=(1, 1), groups=16, bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ELU(alpha=1.0)
    (3): AvgPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0)
    (4): Dropout(p=0.25, inplace=False)
  )
  (separable_conv): Sequential(
    (0): Conv2d(32, 32, kernel_size=(1, 15), stride=(1, 1), padding=(0, 7), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ELU(alpha=1.0)
    (3): AvgPool2d(kernel_size=(1, 8), stride=(1, 8), padding=0)
    (4): Dropout(p=0.25, inplace=False)
  )
  (classfier): Sequential(
    (0): Linear(in_features=736, out_features=2, bias=True)
  )
)


### DeepConvNet

#### Architecture
> Parameters: C = 2, T = 750, N = 2

![DeepConvNet](assets/DeepConvNet.jpg)

In [ ]:
class DeepConvNet(nn.Module):
    first_conv: nn.Sequential
    depthwise_conv: nn.Sequential
    separable_conv: nn.Sequential
    classfier: nn.Sequential

    def __init__(self):
        super(EEGNet, self).__init__()
        self.first_conv = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size = (1, 51), stride = (1, 1), padding = (0, 25), bias = False),
            nn.BatchNorm2d(16, eps = 1e-5, momentum = 0.1, affine = True, track_running_stats = True)
        )
        self.depthwise_conv = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size = (2, 1), stride = (1, 1), groups = 16, bias = False),
            nn.BatchNorm2d(32, eps = 1e-5, momentum = 0.1, affine = True, track_running_stats = True),
            nn.ELU(alpha = 1.0),
            nn.AvgPool2d(kernel_size = (1, 4), stride = (1, 4), padding = 0),
            nn.Dropout(p = 0.25)
        )
        # input: batch, 32, 2, 188
        self.separable_conv = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size = (1, 15), stride = (1, 1), padding = (0, 7), bias = False),
            nn.BatchNorm2d(32, eps = 1e-5, momentum = 0.1, affine = True, track_running_stats = True),
            nn.ELU(alpha = 1.0),
            nn.AvgPool2d(kernel_size = (1, 8), stride = (1, 8), padding = 0),
            nn.Dropout(p = 0.25)
        )
        self.classfier = nn.Sequential(
            nn.Linear(736, 2, bias = True)
        )

    def forward(self, x):
        x = self.first_conv(x)
        x = self.depthwise_conv(x)
        x = self.separable_conv(x)
        return self.classfier(x)

network = EEGNet()
print(network)

## Training

In [34]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)